# Automated ML

In this file we use AutoML to search for the best model that predicts correctly if a couple will divorce or not based on the answer to the questions.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.36.0


## Initialize Workspace
Initialize a workspace object from persisted configuration at config.json

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-177093
aml-quickstarts-177093
southcentralus
f5091c60-1c3c-430f-8d81-d802f6bf2414


## Create an Azure ML experiment
Let's create an experiment


In [3]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'ml-experiment-1'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
ml-experiment-1,quick-starts-ws-177093,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
Let's create a computer node if it doesn't exist or use the existing one.

In [8]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cpuclusterautoml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 1)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded............
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Data
The dataset consists on 54 features that corresponds to questions and 170 samples which corresponds to the answer of 170 couples to those 54 questions. The outcome is the divorced status of the couples based on their answers (1 divorced/0 not divorced)

In [9]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "divorce-ds"
description_text = ""

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key]
        print('dataset found in workspace')

if not found:
        print('dataset NOT found in workspace')
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/joangerard/project-azure-piepline/master/divorce_data.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data, separator=';')        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

dataset NOT found in workspace


,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q46,Q47,Q48,Q49,Q50,Q51,Q52,Q53,Q54,Divorce
count,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,...,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000
mean,1.776471,1.652941,1.764706,1.482353,1.541176,0.747059,0.494118,1.452941,1.458824,1.576471,...,2.552941,2.270588,2.741176,2.382353,2.429412,2.476471,2.517647,2.241176,2.011765,0.494118
std,1.627257,1.468654,1.415444,1.504327,1.632169,0.904046,0.898698,1.546371,1.557976,1.421529,...,1.371786,1.586841,1.137348,1.511587,1.405090,1.260238,1.476537,1.505634,1.667611,0.501442
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,0.000000,0.000000
50%,2.000000,2.000000,2.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,2.000000,...,3.000000,2.000000,3.000000,3.000000,2.000000,3.000000,3.000000,2.000000,2.000000,0.000000
75%,3.000000,3.000000,3.000000,3.000000,3.000000,1.000000,1.000000,3.000000,3.000000,3.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,1.000000
max,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,1.000000


### Review the Dataset Result
Here we can see the first 5 samples which corresponds to couples who divorced.

In [10]:
dataset.take(5).to_pandas_dataframe()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q46,Q47,Q48,Q49,Q50,Q51,Q52,Q53,Q54,Divorce
0,2,2,4,1,0,0,0,0,0,0,...,2,1,3,3,3,2,3,2,1,1
1,4,4,4,4,4,0,0,4,4,4,...,2,2,3,4,4,4,4,2,2,1
2,2,2,2,2,1,3,2,1,1,2,...,3,2,3,1,1,1,2,2,2,1
3,3,2,3,2,3,3,3,3,3,3,...,2,2,3,3,3,3,2,2,2,1
4,2,2,1,1,1,1,0,0,0,0,...,2,1,2,3,2,2,2,1,0,1


## AutoML Configuration
This creates an AutoML settings object and it indicates the label column to be Divorce.

In [15]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Divorce",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep
Here we will use a pipeline to train the data. It will output some metrics and the best model. 

In [16]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [17]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [18]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [19]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [44196ea0][da4086c8-fce6-4f58-ae1e-8f22d6366f37], (This step will run and generate new outputs)
Submitted PipelineRun 34d55c16-a260-4fbe-b524-5f156e0f34b4
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/34d55c16-a260-4fbe-b524-5f156e0f34b4?wsid=/subscriptions/f5091c60-1c3c-430f-8d81-d802f6bf2414/resourcegroups/aml-quickstarts-177093/workspaces/quick-starts-ws-177093&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254


## Run Details

In [20]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [21]:
pipeline_run.wait_for_completion()

PipelineRunId: 34d55c16-a260-4fbe-b524-5f156e0f34b4
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/34d55c16-a260-4fbe-b524-5f156e0f34b4?wsid=/subscriptions/f5091c60-1c3c-430f-8d81-d802f6bf2414/resourcegroups/aml-quickstarts-177093/workspaces/quick-starts-ws-177093&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
PipelineRun Status: Running


StepRunId: 296c76e2-158b-4482-99e3-b14ef348bc65
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/296c76e2-158b-4482-99e3-b14ef348bc65?wsid=/subscriptions/f5091c60-1c3c-430f-8d81-d802f6bf2414/resourcegroups/aml-quickstarts-177093/workspaces/quick-starts-ws-177093&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished

Warnings:
No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/py

'Finished'

## Retrieve the metrics of all child runs

In [22]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/296c76e2-158b-4482-99e3-b14ef348bc65/metrics_data, 1 files out of an estimated total of 1


In [23]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,296c76e2-158b-4482-99e3-b14ef348bc65_1,296c76e2-158b-4482-99e3-b14ef348bc65_3,296c76e2-158b-4482-99e3-b14ef348bc65_4,296c76e2-158b-4482-99e3-b14ef348bc65_0,296c76e2-158b-4482-99e3-b14ef348bc65_15,296c76e2-158b-4482-99e3-b14ef348bc65_18,296c76e2-158b-4482-99e3-b14ef348bc65_10,296c76e2-158b-4482-99e3-b14ef348bc65_12,296c76e2-158b-4482-99e3-b14ef348bc65_13,296c76e2-158b-4482-99e3-b14ef348bc65_14,...,296c76e2-158b-4482-99e3-b14ef348bc65_7,296c76e2-158b-4482-99e3-b14ef348bc65_6,296c76e2-158b-4482-99e3-b14ef348bc65_11,296c76e2-158b-4482-99e3-b14ef348bc65_16,296c76e2-158b-4482-99e3-b14ef348bc65_33,296c76e2-158b-4482-99e3-b14ef348bc65_24,296c76e2-158b-4482-99e3-b14ef348bc65_20,296c76e2-158b-4482-99e3-b14ef348bc65_30,296c76e2-158b-4482-99e3-b14ef348bc65_39,296c76e2-158b-4482-99e3-b14ef348bc65_27
average_precision_score_macro,[0.9967628205128205],[0.99875],[0.99875],[0.9967628205128205],[1.0],[0.99875],[0.99875],[1.0],[1.0],[0.9951781705948374],...,[1.0],[0.9967628205128205],[0.99875],[1.0],[1.0],[0.996475122100122],[1.0],[0.996475122100122],[1.0],[0.9980128205128205]
f1_score_micro,[0.9705882352941178],[0.9764705882352942],[0.9705882352941178],[0.9764705882352942],[0.9764705882352942],[0.9705882352941178],[0.9705882352941178],[0.9764705882352942],[0.9764705882352942],[0.9470588235294117],...,[0.9764705882352942],[0.9705882352941178],[0.9705882352941178],[0.9764705882352942],[0.9764705882352942],[0.9823529411764707],[0.9352941176470588],[0.9764705882352942],[0.9823529411764707],[0.9823529411764707]
balanced_accuracy,[0.9684722222222222],[0.9784722222222223],[0.9684722222222222],[0.9747222222222222],[0.9747222222222222],[0.9684722222222222],[0.9684722222222222],[0.9747222222222222],[0.9747222222222222],[0.9448611111111112],...,[0.9747222222222222],[0.9684722222222222],[0.9684722222222222],[0.9747222222222222],[0.9747222222222222],[0.9802777777777777],[0.9349999999999999],[0.9740277777777777],[0.9802777777777777],[0.9802777777777777]
norm_macro_recall,[0.9369444444444444],[0.9569444444444445],[0.9369444444444444],[0.9494444444444443],[0.9494444444444443],[0.9369444444444444],[0.9369444444444444],[0.9494444444444443],[0.9494444444444443],[0.8897222222222222],...,[0.9494444444444443],[0.9369444444444444],[0.9369444444444444],[0.9494444444444443],[0.9494444444444443],[0.9605555555555554],[0.8699999999999999],[0.9480555555555554],[0.9605555555555554],[0.9605555555555554]
AUC_macro,[0.9969444444444445],[0.9986111111111111],[0.9986111111111111],[0.9969444444444445],[1.0],[0.9986111111111111],[0.9986111111111111],[1.0],[1.0],[0.9944444444444442],...,[1.0],[0.9969444444444445],[0.9986111111111111],[1.0],[1.0],[0.9966666666666667],[1.0],[0.9966666666666667],[1.0],[0.9983333333333334]
precision_score_weighted,[0.9726747109100049],[0.9781045751633988],[0.9726747109100049],[0.9792106586224232],[0.9792106586224232],[0.9726747109100049],[0.9726747109100049],[0.9792106586224232],[0.9792106586224232],[0.9509286791639733],...,[0.9792106586224232],[0.9726747109100049],[0.9726747109100049],[0.9792106586224232],[0.9792106586224232],[0.9844394167923578],[0.9487516568398922],[0.9791452991452992],[0.9844394167923578],[0.9844394167923578]
recall_score_micro,[0.9705882352941178],[0.9764705882352942],[0.9705882352941178],[0.9764705882352942],[0.9764705882352942],[0.9705882352941178],[0.9705882352941178],[0.9764705882352942],[0.9764705882352942],[0.9470588235294117],...,[0.9764705882352942],[0.9705882352941178],[0.9705882352941178],[0.9764705882352942],[0.9764705882352942],[0.9823529411764707],[0.9352941176470588],[0.9764705882352942],[0.9823529411764707],[0.9823529411764707]
matthews_correlation,[0.9386123000100224],[0.9526785151252903],[0.9386123000100224],[0.9511123000100223],[0.9511123000100223],[0.9386123000100224],[0.9386123000100224],[0.9511123000100223],[0.9511123000100223],[0.8933996344249813],...,[0.9511123000100223],[0.9386123000100224],[0.9386123000100224],[0.9511123000100223],[0.9511123000100223],[0.9622234111211334],[0.879046503881599],[0.95096460786

## Best Model

In [24]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/296c76e2-158b-4482-99e3-b14ef348bc65/model_data, 1 files out of an estimated total of 1


In [25]:
# Save the best model
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 RandomForestClassifier(bootstrap=False, ccp_alpha=0.0,
                                        class_weight=None, criterion='entropy',
                                        max_depth=None, max_features=0.1,
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=0.01,
                                        min_samples_split=0.01,
                                        min_

In [26]:
best_model.steps

[('datatransformer',
  DataTransformer(
      task='classification',
      is_onnx_compatible=False,
      enable_feature_sweeping=True,
      enable_dnn=False,
      force_text_dnn=False,
      feature_sweeping_timeout=86400,
      featurization_config=None,
      is_cross_validation=True,
      feature_sweeping_config={}
  )),
 ('StandardScalerWrapper',
  StandardScalerWrapper(
      copy=True,
      with_mean=False,
      with_std=False
  )),
 ('RandomForestClassifier',
  RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                         criterion='entropy', max_depth=None, max_features=0.1,
                         max_leaf_nodes=None, max_samples=None,
                         min_impurity_decrease=0.0, min_impurity_split=None,
                         min_samples_leaf=0.01, min_samples_split=0.01,
                         min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
                         oob_score=False, random_state=None, verbose=0,

In [27]:
### Explore models and metrics
from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0]+ ' - ')
        elif hasattr(step[1], '_base_learners') and hasattr(step[1], '_meta_learner'):
            print("\nMeta Learner")
            pprint(step[1]._meta_learner)
            print()
            for estimator in step[1]._base_learners:
                print_model(estimator[1], estimator[0]+ ' - ')
        else:
            pprint(step[1].get_params())
            print()
            
print_model(best_model)

datatransformer
{'enable_dnn': False,
 'enable_feature_sweeping': True,
 'feature_sweeping_config': {},
 'feature_sweeping_timeout': 86400,
 'featurization_config': None,
 'force_text_dnn': False,
 'is_cross_validation': True,
 'is_onnx_compatible': False,
 'observer': None,
 'task': 'classification',
 'working_dir': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook177093/code/divorce-prediction-azure-ml'}

StandardScalerWrapper
{'class_name': 'StandardScaler',
 'copy': True,
 'module_name': 'sklearn.preprocessing._data',
 'with_mean': False,
 'with_std': False}

RandomForestClassifier
{'bootstrap': False,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'entropy',
 'max_depth': None,
 'max_features': 0.1,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 0.01,
 'min_samples_split': 0.01,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 50,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': 

In [49]:
best_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 RandomForestClassifier(bootstrap=False, ccp_alpha=0.0,
                                        class_weight=None, criterion='entropy',
                                        max_depth=None, max_features=0.1,
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=0.01,
                                        min_samples_split=0.01,
                                        min_

In [51]:
from azureml.train.automl.run import AutoMLRun

run_id = ''
for step in pipeline_run.get_steps():
    run_id = step.id
automl_run = AutoMLRun(experiment=experiment, run_id=run_id)
best_run, fitted_model = automl_run.get_output()
print(best_run)
print_model(fitted_model)

Package:azureml-automl-runtime, training version:1.37.0, current version:1.36.0
Package:azureml-core, training version:1.37.0, current version:1.36.0.post2
Package:azureml-dataprep, training version:2.25.0, current version:2.24.4
Package:azureml-dataprep-rslex, training version:2.1.0, current version:2.0.3
Package:azureml-dataset-runtime, training version:1.37.0, current version:1.36.0
Package:azureml-defaults, training version:1.37.0, current version:1.36.0
Package:azureml-interpret, training version:1.37.0, current version:1.36.0
Package:azureml-mlflow, training version:1.37.0, current version:1.36.0
Package:azureml-pipeline-core, training version:1.37.0, current version:1.36.0
Package:azureml-responsibleai, training version:1.37.0, current version:1.36.0
Package:azureml-telemetry, training version:1.37.0, current version:1.36.0
Package:azureml-train-automl-client, training version:1.37.0, current version:1.36.0
Package:azureml-train-automl-runtime, training version:1.37.0, current v

Run(Experiment: ml-experiment-1,
Id: 296c76e2-158b-4482-99e3-b14ef348bc65_17,
Type: azureml.scriptrun,
Status: Completed)
datatransformer
{'enable_dnn': False,
 'enable_feature_sweeping': True,
 'feature_sweeping_config': {},
 'feature_sweeping_timeout': 86400,
 'featurization_config': None,
 'force_text_dnn': False,
 'is_cross_validation': True,
 'is_onnx_compatible': False,
 'observer': None,
 'task': 'classification',
 'working_dir': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook177093/code/divorce-prediction-azure-ml'}

StandardScalerWrapper
{'class_name': 'StandardScaler',
 'copy': True,
 'module_name': 'sklearn.preprocessing._data',
 'with_mean': False,
 'with_std': False}

RandomForestClassifier
{'bootstrap': False,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'entropy',
 'max_depth': None,
 'max_features': 0.1,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 0.01,
 'min_samples

## Register best model

In [ ]:
# Register it
from azureml.core.run import Run

run = Run.get_context()
pipeline_run.upload_file("./outputs/model.pkl", best_model_output._path_on_datastore)
print(pipeline_run.get_file_names())
model = pipeline_run.register_model(model_name="auto-ml-model", model_path="./outputs/model.pkl")